In [1]:
# Install PuLP (for optimization) – only needed once
!pip install pulp

import pandas as pd
import numpy as np
import pulp as pl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.4 MB/s eta 0:00:00


In [2]:
# Load the grid data we exported from the other notebook
grid_df = pd.read_csv("grid_for_optimization.csv")

print("Grid shape:", grid_df.shape)
print(grid_df.head())
print("\nColumns:", grid_df.columns.tolist())


Grid shape: (5270, 10)
   cell_id   x_coord    y_coord  collision_count  camera_count  \
0        0  609804.0  4827231.0                0             0   
1        1  609804.0  4827731.0                0             0   
2        2  609804.0  4828231.0                0             0   
3        3  609804.0  4828731.0                0             0   
4        4  609804.0  4829231.0                0             0   

   mean_dist_to_camera  mean_precip  mean_snow  downtown    lambda  
0                  0.0          0.0        0.0         0  0.309433  
1                  0.0          0.0        0.0         0  0.309433  
2                  0.0          0.0        0.0         0  0.309433  
3                  0.0          0.0        0.0         0  0.309433  
4                  0.0          0.0        0.0         0  0.309433  

Columns: ['cell_id', 'x_coord', 'y_coord', 'collision_count', 'camera_count', 'mean_dist_to_camera', 'mean_precip', 'mean_snow', 'downtown', 'lambda']


In [3]:
# === NEW WEIGHT DEFINITION WITH WEATHER (+ model risk) ===

df = grid_df.copy()

# 1) Base risk: historical collisions in that 500m cell
base = df["collision_count"].astype(float)

# 2) Downtown premium (20% extra weight for downtown cells)
downtown_factor = 1 + 0.2 * df["downtown"]

# 3) Weather factors: normalize mean_precip and mean_snow to [0,1]
precip = df["mean_precip"].astype(float)
snow   = df["mean_snow"].astype(float)

def safe_min_max(series):
    s_min, s_max = series.min(), series.max()
    if s_max > s_min:
        return (series - s_min) / (s_max - s_min)
    else:
        return 0 * series  # all zeros if no variation

precip_norm = safe_min_max(precip)   # between 0 and 1
snow_norm   = safe_min_max(snow)     # between 0 and 1

# 4) Model-predicted intensity from lambda (Negative Binomial)
lambda_term = df["lambda"].astype(float)
lambda_norm = safe_min_max(lambda_term)

# 5) Combine them:
#    - weather_bump: cells with worse weather get more weight
#    - model_bump: cells with higher predicted collisions get more weight
weather_bump = 0.5 * (precip_norm + snow_norm)
model_bump   = 0.5 * lambda_norm

raw_weight = base * (1 + weather_bump + model_bump)

# 6) Apply downtown premium
df["weight"] = raw_weight * downtown_factor

# 7) Make sure weights are non-negative
df["weight"] = df["weight"].clip(lower=0)

print(df[["cell_id", "collision_count", "downtown",
          "mean_precip", "mean_snow", "lambda", "weight"]].head(10))
print("\nTotal weight:", df["weight"].sum())


   cell_id  collision_count  downtown  mean_precip  mean_snow    lambda  \
0        0                0         0          0.0        0.0  0.309433   
1        1                0         0          0.0        0.0  0.309433   
2        2                0         0          0.0        0.0  0.309433   
3        3                0         0          0.0        0.0  0.309433   
4        4                0         0          0.0        0.0  0.309433   
5        5                0         0          0.0        0.0  0.309433   
6        6                0         0          0.0        0.0  0.309433   
7        7                0         0          0.0        0.0  0.309433   
8        8                0         0          0.0        0.0  0.309433   
9        9                0         0          0.0        0.0  0.309433   

   weight  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  
5     0.0  
6     0.0  
7     0.0  
8     0.0  
9     0.0  

Total weight: 751111.203025734


In [4]:
#5.2 demand cells
# Demand set I: all grid cells
demand_df = df[["cell_id", "x_coord", "y_coord", "weight"]].copy()
demand_df = demand_df.rename(columns={"cell_id": "demand_id"})

print("Number of demand cells:", len(demand_df))
print(demand_df.head())


Number of demand cells: 5270
   demand_id   x_coord    y_coord  weight
0          0  609804.0  4827231.0     0.0
1          1  609804.0  4827731.0     0.0
2          2  609804.0  4828231.0     0.0
3          3  609804.0  4828731.0     0.0
4          4  609804.0  4829231.0     0.0


In [7]:
#5.3 Define candidate camera sites (top high-risk cells)

# Choose how many candidate sites we allow (you can change this later)
TOP_N_CANDIDATES = 1500  # try 1500 first

# Sort cells by risk weight descending
candidates_df = df.sort_values("weight", ascending=False).head(TOP_N_CANDIDATES)

candidates_df = candidates_df[["cell_id", "x_coord", "y_coord"]].copy()
candidates_df = candidates_df.rename(columns={"cell_id": "site_id"})

print("Number of candidate sites:", len(candidates_df))
print(candidates_df.head())


Number of candidate sites: 1500
      site_id   x_coord    y_coord
2617     2617  630804.0  4833731.0
2556     2556  630304.0  4834231.0
2619     2619  630804.0  4834731.0
2056     2056  626304.0  4832231.0
2369     2369  628804.0  4833731.0


In [8]:
import numpy as np
import pulp as pl

def run_camera_optimization(K, RADIUS, scenario_name=""):
    """
    Run the max-coverage camera model for a given
    number of cameras K and coverage radius RADIUS (in meters).

    Uses global demand_df and candidates_df that you already built.
    Returns: solution_sites_df (chosen sites) and coverage_percent.
    """

    # --- Build coverage pairs (i,j) for this radius ---
    demand_ids = demand_df["demand_id"].tolist()
    site_ids   = candidates_df["site_id"].tolist()

    demand_x = demand_df.set_index("demand_id")["x_coord"]
    demand_y = demand_df.set_index("demand_id")["y_coord"]
    site_x   = candidates_df.set_index("site_id")["x_coord"]
    site_y   = candidates_df.set_index("site_id")["y_coord"]

    pairs = []
    for i in demand_ids:
        xi = demand_x[i]
        yi = demand_y[i]
        for j in site_ids:
            dx = xi - site_x[j]
            dy = yi - site_y[j]
            dist = np.sqrt(dx*dx + dy*dy)
            if dist <= RADIUS:
                pairs.append({"demand_id": i, "site_id": j})

    coverage_df = pd.DataFrame(pairs)
    print(f"\n[{scenario_name}] RADIUS = {RADIUS} m")
    print("Number of (i,j) coverage pairs:", len(coverage_df))

    # --- Build cover dictionary a_ij ---
    covers = {(row.demand_id, row.site_id): 1
              for row in coverage_df.itertuples(index=False)}

    # --- Weights w_i (already in df, joined by cell_id / demand_id) ---
    w = demand_df.set_index("demand_id")["weight"].to_dict()

    # --- Create max-coverage model in PuLP ---
    m = pl.LpProblem(f"Camera_Placement_{scenario_name}", pl.LpMaximize)

    # Decision variables
    x = pl.LpVariable.dicts("x", site_ids, lowBound=0, upBound=1, cat="Binary")
    y = pl.LpVariable.dicts("y", demand_ids, lowBound=0, upBound=1, cat="Binary")

    # Objective: maximize total weighted coverage
    m += pl.lpSum(w[i] * y[i] for i in demand_ids), "Total_Weighted_Coverage"

    # Coverage constraints: y_i = 1 only if at least one chosen camera covers i
    for i in demand_ids:
        sites_that_cover_i = [j for (ii, j) in covers.keys() if ii == i]
        if sites_that_cover_i:
            m += pl.lpSum(x[j] for j in sites_that_cover_i) >= y[i], f"Coverage_{i}"
        else:
            # no site can cover this cell -> y_i forced to 0
            m += y[i] == 0, f"NoCoverage_{i}"

    # Camera budget
    m += pl.lpSum(x[j] for j in site_ids) <= K, "Camera_Budget"

    # Solve
    m.solve(pl.PULP_CBC_CMD(msg=False))
    print("Status:", pl.LpStatus[m.status])

    # Extract chosen sites
    chosen_sites = [j for j in site_ids if x[j].value() is not None and x[j].value() > 0.5]
    solution_sites_df = candidates_df[candidates_df["site_id"].isin(chosen_sites)].copy()

    # Compute coverage percent
    covered_cells = sum(1 for i in demand_ids if y[i].value() is not None and y[i].value() > 0.5)
    total_cells   = len(demand_ids)
    coverage_percent = 100.0 * covered_cells / total_cells if total_cells > 0 else 0.0

    print(f"Number of cameras K = {K}")
    print(f"Covered cells: {covered_cells} / {total_cells} ({coverage_percent:.2f}%)")

    return solution_sites_df, coverage_percent


In [9]:
scenarios = [
    {"name": "K150_R500", "K": 150, "RADIUS": 500},
    {"name": "K250_R500", "K": 250, "RADIUS": 500},
    {"name": "K150_R250", "K": 150, "RADIUS": 250},
    {"name": "K250_R250", "K": 250, "RADIUS": 250},
]

results = []

for s in scenarios:
    print("\n==============================")
    print(f"Running scenario: {s['name']}")
    sol_df, cov_pct = run_camera_optimization(
        K=s["K"],
        RADIUS=s["RADIUS"],
        scenario_name=s["name"],
    )

    # save chosen sites for this scenario
    out_name = f"optimal_camera_sites_{s['name']}.csv"
    sol_df.to_csv(out_name, index=False)
    print(f"Saved {out_name}")

    results.append({
        "scenario": s["name"],
        "K": s["K"],
        "RADIUS": s["RADIUS"],
        "coverage_percent": cov_pct,
    })

pd.DataFrame(results)



Running scenario: K150_R500

[K150_R500] RADIUS = 500 m
Number of (i,j) coverage pairs: 7498
Status: Optimal
Number of cameras K = 150
Covered cells: 728 / 5270 (13.81%)
Saved optimal_camera_sites_K150_R500.csv

Running scenario: K250_R500

[K250_R500] RADIUS = 500 m
Number of (i,j) coverage pairs: 7498
Status: Optimal
Number of cameras K = 250
Covered cells: 1207 / 5270 (22.90%)
Saved optimal_camera_sites_K250_R500.csv

Running scenario: K150_R250

[K150_R250] RADIUS = 250 m
Number of (i,j) coverage pairs: 1500
Status: Optimal
Number of cameras K = 150
Covered cells: 150 / 5270 (2.85%)
Saved optimal_camera_sites_K150_R250.csv

Running scenario: K250_R250

[K250_R250] RADIUS = 250 m
Number of (i,j) coverage pairs: 1500
Status: Optimal
Number of cameras K = 250
Covered cells: 250 / 5270 (4.74%)
Saved optimal_camera_sites_K250_R250.csv


,scenario,K,RADIUS,coverage_percent
0,K150_R500,150,500,13.814042
1,K250_R500,250,500,22.903226
2,K150_R250,150,250,2.846300
3,K250_R250,250,250,4.743833


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['K'].plot(kind='hist', bins=20, title='K')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['RADIUS'].plot(kind='hist', bins=20, title='RADIUS')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_17['scenario'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_17, x='K', y='scenario', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_18['scenario'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_18, x='RADIUS', y='scenario', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['scenario'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='coverage_percent', y='scenario', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [10]:
#6.1 Coverage Table
# === Step 6: Build coverage matrix a_ij ===

RADIUS = 500.0  # metres, same as  grid size

demand_ids = demand_df["demand_id"].tolist()
site_ids = candidates_df["site_id"].tolist()

# quick lookup dicts for coordinates
demand_x = dict(zip(demand_df["demand_id"], demand_df["x_coord"]))
demand_y = dict(zip(demand_df["demand_id"], demand_df["y_coord"]))
site_x   = dict(zip(candidates_df["site_id"], candidates_df["x_coord"]))
site_y   = dict(zip(candidates_df["site_id"], candidates_df["y_coord"]))

pairs = []

for i in demand_ids:
    xi = demand_x[i]
    yi = demand_y[i]
    for j in site_ids:
        dx = xi - site_x[j]
        dy = yi - site_y[j]
        dist = np.sqrt(dx*dx + dy*dy)
        if dist <= RADIUS:
            pairs.append({"demand_id": i, "site_id": j})

coverage_df = pd.DataFrame(pairs)

print("Number of demand cells:", len(demand_ids))
print("Number of candidate sites:", len(site_ids))
print("Number of (i,j) coverage pairs:", len(coverage_df))
print(coverage_df.head())


Number of demand cells: 5270
Number of candidate sites: 1500
Number of (i,j) coverage pairs: 7498
   demand_id  site_id
0         33       95
1         35       97
2         35       36
3         36       36
4         37       36


In [11]:
# === Step 7.1: Prepare parameters for the optimization model ===

# Demand weights w_i (risk in each cell)
w = dict(zip(demand_df["demand_id"], demand_df["weight"]))

# Coverage indicators a_ij: we only store the (i,j) pairs where coverage=1
covers = {(row.demand_id, row.site_id): 1 for row in coverage_df.itertuples(index=False)}

print("Example weights (first 5):", list(w.items())[:5])
print("Example coverage pairs (first 5):", list(covers.items())[:5])

# Choose how many cameras we are allowed to place (this is a policy choice)
K = 150  # you can later try 30, 50, 70, etc.
print("\nMaximum number of cameras K =", K)


Example weights (first 5): [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0), (4, 0.0)]
Example coverage pairs (first 5): [((33, 95), 1), ((35, 97), 1), ((35, 36), 1), ((36, 36), 1), ((37, 36), 1)]

Maximum number of cameras K = 150


In [ ]:
# === Step 7.2: Define the maximum coverage model in PuLP ===

import pulp as pl  # in case it wasn't imported earlier

# Create model
m = pl.LpProblem("Camera_Placement_MaxCoverage", pl.LpMaximize)

# Decision variables:
# x_j = 1 if we place a camera at candidate site j
x = pl.LpVariable.dicts("x", candidates_df["site_id"].tolist(),
                        lowBound=0, upBound=1, cat="Binary")

# y_i = 1 if demand cell i is covered by at least one chosen camera
y = pl.LpVariable.dicts("y", demand_df["demand_id"].tolist(),
                        lowBound=0, upBound=1, cat="Binary")

# Objective: maximize total weighted coverage sum_i w_i * y_i
m += pl.lpSum(w[i] * y[i] for i in demand_df["demand_id"]), "Total_Weighted_Coverage"

# Coverage constraints: for each demand cell i,
# sum_j a_ij * x_j >= y_i  (if there are any j that can cover it)
for i in demand_df["demand_id"]:
    sites_that_cover_i = [j for (ii, j) in covers.keys() if ii == i]
    if sites_that_cover_i:
        m += pl.lpSum(x[j] for j in sites_that_cover_i) >= y[i], f"Coverage_{i}"
    else:
        # No candidate covers this cell -> y_i must be 0
        m += y[i] == 0, f"NoCoverage_{i}"

# Camera budget: total number of cameras <= K
m += pl.lpSum(x[j] for j in candidates_df["site_id"].tolist()) <= K, "Camera_Budget"

print("Model has", len(m.variables()), "variables and", len(m.constraints), "constraints.")


Model has 6770 variables and 5271 constraints.


In [ ]:
# 8.1 Solve optimization model
# === Step 8.1: Solve the maximum coverage model ===

result_status = m.solve(pl.PULP_CBC_CMD(msg=True))

print("Solver status:", pl.LpStatus[m.status])
print("Optimal objective value (total weighted coverage):", pl.value(m.objective))


Solver status: Optimal
Optimal objective value (total weighted coverage): 511281.57625842746


In [ ]:
#8.2 compute how much risk is covered.
# === Step 8.2: Compute coverage percentage ===

total_weight = demand_df["weight"].sum()
covered_weight = 0.0

for i in demand_df["demand_id"]:
    if y[i].value() is not None and y[i].value() > 0.5:
        covered_weight += w[i]

coverage_percent = 100 * covered_weight / total_weight

print("Total risk weight:", total_weight)
print("Covered risk weight:", covered_weight)
print("Coverage percent: {:.2f}%".format(coverage_percent))


Total risk weight: 751111.203025734
Covered risk weight: 511281.57625842746
Coverage percent: 68.07%


In [ ]:
# === Step 8.3: Which sites get cameras? ===

chosen_sites = [
    j for j in candidates_df["site_id"]
    if x[j].value() is not None and x[j].value() > 0.5
]

print("Number of cameras chosen:", len(chosen_sites))
print("First 20 chosen site IDs:", chosen_sites[:20])

solution_sites_df = candidates_df[candidates_df["site_id"].isin(chosen_sites)].copy()
print("\nSample of chosen sites:")
print(solution_sites_df.head())


Number of cameras chosen: 150
First 20 chosen site IDs: [3072, 2616, 2680, 2271, 2956, 2493, 1215, 4073, 3321, 1581, 3014, 4388, 2429, 2557, 2520, 3757, 2994, 3516, 1331, 4505]

Sample of chosen sites:
      site_id   x_coord    y_coord
3072     3072  634304.0  4844231.0
2616     2616  630804.0  4833231.0
2680     2680  631304.0  4834231.0
2271     2271  627804.0  4846731.0
2956     2956  633304.0  4848231.0


In [ ]:
# === Step 8.4: Save the optimal camera locations to CSV ===

out_name = f"optimal_camera_sites_K{K}.csv"
solution_sites_df.to_csv(out_name, index=False)
print(f"Saved {out_name} in this Colab session.")


Saved optimal_camera_sites_K150.csv in this Colab session.


In [ ]:
!pip install geopandas shapely folium


In [ ]:
import geopandas as gpd
from shapely.geometry import Point

# solution_sites_df already has: site_id, x_coord, y_coord
print(solution_sites_df.head())

# Create GeoDataFrame in the projected CRS (same as your grid: EPSG:32617)
gdf_sites = gpd.GeoDataFrame(
    solution_sites_df.copy(),
    geometry=gpd.points_from_xy(solution_sites_df["x_coord"], solution_sites_df["y_coord"]),
    crs="EPSG:32617"
)

# Reproject to WGS84 lat/lon for mapping
gdf_sites_4326 = gdf_sites.to_crs(epsg=4326)

# Extract lat/lon columns
gdf_sites_4326["lat"] = gdf_sites_4326.geometry.y
gdf_sites_4326["lon"] = gdf_sites_4326.geometry.x

print(gdf_sites_4326[["site_id", "lat", "lon"]].head())


      site_id   x_coord    y_coord
3072     3072  634304.0  4844231.0
2616     2616  630804.0  4833231.0
2680     2680  631304.0  4834231.0
2271     2271  627804.0  4846731.0
2956     2956  633304.0  4848231.0
      site_id        lat        lon
3072     3072  43.738951 -79.332152
2616     2616  43.640572 -79.378270
2680     2680  43.649485 -79.371830
2271     2271  43.762601 -79.412246
2956     2956  43.775132 -79.343572


In [ ]:
import pandas as pd
import geopandas as gpd

# === Load existing speed cameras from uploaded file ===
speed_cameras = pd.read_csv("/content/speed_cameras_clean.csv")
# you can also just do: speed_cameras = pd.read_csv("speed_cameras_clean.csv")

print(speed_cameras.head())


existing_gdf = gpd.GeoDataFrame(
    speed_cameras.copy(),
    geometry=gpd.points_from_xy(speed_cameras["lon"], speed_cameras["lat"]),
    crs="EPSG:4326"
)
existing_gdf["lat"] = existing_gdf.geometry.y
existing_gdf["lon"] = existing_gdf.geometry.x

print("Existing cameras:", len(existing_gdf))


         lon        lat status_clean  ward_num  \
0 -79.567451  43.713609       active         1   
1 -79.550510  43.700973       active         1   
2 -79.561386  43.728553       active         1   
3 -79.597523  43.748744       active         1   
4 -79.553404  43.722559       active         1   

                                    location  fid  
0        Kipling Ave. North of Rexdale Blvd.    1  
1   St. Andrews Blvd. West of Islington Ave.    2  
2     Islington Ave. North of Fordwich Cres.    3  
3  Martin Grove Rd. South of Silverstone Dr.    4  
4           Golfdown Dr. East of Turpin Ave.    5  
Existing cameras: 198


In [ ]:
# === Before/After map of cameras ===
import folium

# Center map on Toronto using existing cameras
center_lat = existing_gdf["lat"].mean()
center_lon = existing_gdf["lon"].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    tiles="CartoDB positron"
)

# --- Layer 1: existing cameras (BEFORE, red) ---
layer_existing = folium.FeatureGroup(name="Existing speed cameras")

for _, row in existing_gdf.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=4,
        color="red",
        fill=True,
        fill_opacity=0.8,
        popup=row.get("location", "Existing camera")
    ).add_to(layer_existing)

layer_existing.add_to(m)

# --- Layer 2: optimized cameras (AFTER, blue) ---
# gdf_sites_4326 already has lat/lon from your first cell
layer_opt = folium.FeatureGroup(name="Optimized camera sites")

for _, row in gdf_sites_4326.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=4,
        color="blue",
        fill=True,
        fill_opacity=0.8,
        popup=f"Optimized site {row['site_id']}"
    ).add_to(layer_opt)

layer_opt.add_to(m)

# Layer control to toggle BEFORE / AFTER
folium.LayerControl().add_to(m)

# --- Add legend (red = previous, blue = current/optimized) ---
from folium import Element

legend_html = """
<div style="
    position: fixed;
    bottom: 50px;
    left: 50px;
    width: 190px;
    z-index: 9999;
    background-color: white;
    border: 2px solid grey;
    border-radius: 5px;
    padding: 10px;
    font-size: 14px;
">
<b>Legend</b><br>
<span style="color:red;">&#9679;</span> Existing cameras<br>
<span style="color:blue;">&#9679;</span> Optimized cameras
</div>
"""

m.get_root().html.add_child(Element(legend_html))

m



In [ ]:
m.save("optimal_camera_map_K150_R500.html")
print("Saved optimal_camera_map_K150.html")


Saved optimal_camera_map_K150.html


We formulated a maximum-coverage location model on a 500m × 500m grid over Toronto. Each grid cell is treated as a demand area with a risk weight equal to its historical collision count, slightly increased in downtown cells. Candidate camera locations are restricted to the 1,500 highest-risk cells. A camera is assumed to influence collisions within a 500m radius, so a cell is considered “covered” if at least one selected camera lies within this distance. The model chooses at most
K camera sites (here K=150) to maximize the total risk weight of covered cells. In the solved scenario, the optimal configuration of 150 cameras covers approximately 68% of the total collision risk, indicating that a relatively small number of strategically placed cameras can protect the majority of high-risk areas while leaving some lower-risk cells uncovered.

### Weights

For each grid cell (demand area) i we define a risk weight w_i:

- Base risk is the historical number of collisions in that cell:
  w_i_base = collision_count_i

- Downtown cells are slightly upweighted to reflect higher policy priority:
  w_i = 1.2 × collision_count_i   if downtown_i = 1
  w_i = 1.0 × collision_count_i   if downtown_i = 0

So the weights represent “collision risk”, with a 20% bonus weight for downtown cells.

(Other variables such as mean_precip, mean_snow, and lambda are not directly used as weights in this optimization model.)

### Decision variables

We use two sets of binary decision variables:

1. Camera location variables x_j
   - Defined for each candidate site j (top high-risk grid cells where a camera could be placed).
   - Interpretation:
       x_j = 1  → place a speed camera at candidate site j
       x_j = 0  → do not place a camera at site j

2. Coverage variables y_i
   - Defined for each demand cell i (every grid cell in the city).
   - Interpretation:
       y_i = 1  → cell i is covered by at least one selected camera within 500 m
       y_i = 0  → cell i is not covered

The objective of the model is:

   Maximize  Σ_i w_i · y_i

That is, choose the camera locations (x_j) so that the total risk weight of covered cells (where y_i = 1) is as large as possible, subject to coverage and budget constraints.
